In [ ]:
def source_predict(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"]}
    :return:
    '''
    src_source_info = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    # 根据角色创建时间类型分组
    columns = ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num","create_role_cost",
        "create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d","label"]
    source_predict = pd.DataFrame(columns=columns)

    src_source_info_0 = src_source_info[src_source_info['data_win'] == 0]
    src_source_info_1_7 = src_source_info[src_source_info['data_win'] != 0]
    drop_features_0 = ["channel_id","source_id","source_run_datetime","create_time","media_id","game_id","create_role_pay_num","create_role_pay_sum","create_role_retain_1d"]
    drop_features_1_7 = ["channel_id", "source_id", "source_run_datetime", "create_time", "media_id", "game_id","create_role_pay_num", "create_role_pay_sum"]
    if src_source_info_0.shape[0] != 0:
        src_source_info_0['label'] = source_lgb_b_cv_0.predict(src_source_info_0.drop(drop_features_0, axis=1))
        src_source_info_0['label'] = src_source_info_0['label'].apply(lambda x: 1 if x >= 0.7 else 0)
    if src_source_info_1_7.shape[0] != 0:
        src_source_info_1_7['label'] = source_lgb_b_cv.predict(src_source_info_1_7.drop(drop_features_1_7, axis=1))
        src_source_info_1_7['label'] = src_source_info_1_7['label'].apply(lambda x: 1 if x >= 0.5 else 0)

    # 数据输出
    source_predict = source_predict.append(src_source_info_0).append(src_source_info_1_7)
    source_predict['label'] = source_predict['label'].astype(int)

    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}